In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.special import boxcox, inv_boxcox

# Import Data for V1.1 Feature Selection

In [2]:
X_train= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/x_Train.csv')
X_test= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/x_Test.csv')
y_train= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/y_Train.csv')
y_test= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/y_Test.csv')
finalDF = pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/FinalTest.csv')

In [3]:
columnList1 = []
for i in finalDF.columns:
    columnList1.append(i)
    for i in X_train.columns:
        columnList1.append(i)
        for i in X_test.columns:
            columnList1.append(i)
columnSet = set(columnList1)


In [4]:
columnList2 = list(columnSet)
len(columnList2)

196

In [5]:
addList = []
for i in columnList2:
    if i not in X_train.columns:
        addList.append(i)
X_train[addList] = 0
addList = []
for i in columnList2:
    if i not in finalDF.columns:
        addList.append(i)
finalDF[addList] = 0
addList = []
for i in columnList2:
    if i not in X_test.columns:
        addList.append(i)
X_test[addList] = 0

In [6]:
len(X_test.columns)

196

# Use BoxCox to transform target variable

In [10]:
y_train, y_trainLambda = stats.boxcox(y_train.SalePrice)

In [14]:
y_test = stats.boxcox(y_test.SalePrice, lmbda = y_trainLambda)

In [16]:
y_train = pd.DataFrame(y_train)
y_train.columns = ['SalePrice']
y_test = pd.DataFrame(y_test)
y_test.columns = ['SalePrice']

In [17]:
y_train

,SalePrice
0,8.893344
1,9.103165
2,8.717503
3,8.526283
4,8.886665
...,...
1004,8.756363
1005,8.794272
1006,9.126915
1007,8.923840


# Fit an ElasticNet Model With GridSearch

In [18]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [100]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
parametersGrid = {"max_iter": [1, 5, 10],
                "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                "l1_ratio": np.arange(0.0, 1.0, 0.1)}
eNet = ElasticNet(normalize=True)
gridEn = GridSearchCV(eNet, parametersGrid, scoring='r2', cv=10, n_jobs = -1)
gridEn.fit(X_train, y_train)
predEn = gridEn.predict(X_test)
    

In [101]:
PredGridEn = pd.concat([pd.Series(predEn), y_test], axis = 1)
PredGridEn.columns = ['Predicted', 'Actual']
PredGridEn


,Predicted,Actual
0,8.895371,8.887317
1,8.899785,8.756363
2,9.086227,9.112309
3,8.800891,8.849539
4,8.817136,8.760285
...,...,...
428,8.674661,8.759307
429,8.948511,8.943177
430,8.844391,8.799720
431,8.725798,8.686831


In [102]:
PredGrid['Predicted'] = inv_boxcox(PredGrid['Predicted'], y_trainLambda)
PredGrid['Actual'] = inv_boxcox(PredGrid['Actual'], y_trainLambda)

## Lets score ourselves

In [27]:
PredGrid['Difference']=(PredGrid['Predicted'] - PredGrid['Actual'])**2
PredGrid.Difference.sum()


343933218287.4431

In [28]:
PredGrid.Difference.mean()**.5


28183.382418434947

In [29]:
grid.score(X_test, y_test)

0.8681215973213532

In [30]:
grid.get_params()

{'cv': 10,
 'error_score': nan,
 'estimator__alpha': 1.0,
 'estimator__copy_X': True,
 'estimator__fit_intercept': True,
 'estimator__l1_ratio': 0.5,
 'estimator__max_iter': 1000,
 'estimator__normalize': True,
 'estimator__positive': False,
 'estimator__precompute': False,
 'estimator__random_state': None,
 'estimator__selection': 'cyclic',
 'estimator__tol': 0.0001,
 'estimator__warm_start': False,
 'estimator': ElasticNet(normalize=True),
 'n_jobs': None,
 'param_grid': {'max_iter': [1, 5, 10],
  'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
  'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': 'r2',
 'verbose': 0}

## Lets try a Lasso Model

In [31]:
from sklearn.linear_model import Lasso

In [32]:
logit = Lasso()

In [33]:
logit.fit(X_train, y_train)

Lasso()

In [34]:
LassoDF = pd.DataFrame(logit.coef_)
G = pd.concat([pd.Series(X_train.columns), LassoDF], axis = 1)
G.columns = ['Feature', 'Lasso Coefficient']
G.sort_values('Lasso Coefficient')

,Feature,Lasso Coefficient
0,LotFrontage,0.000000e+00
125,Exterior2nd_Wd Sdng,-0.000000e+00
126,Exterior2nd_Wd Shng,-0.000000e+00
127,MasVnrType_BrkFace,0.000000e+00
128,MasVnrType_None,-0.000000e+00
...,...,...
195,Unnamed: 0,0.000000e+00
1,LotArea,2.399326e-07
2,MasVnrArea,5.440483e-05
10,OutSq,1.209200e-04


In [35]:
logit.score(X_test, y_test)

0.7069754075529064

In [38]:
LassoPred = inv_boxcox(logit.predict(X_test), y_trainLambda)

In [39]:
LassoPred

array([150648.66969009, 215524.05250025, 236758.91106367, 149574.23990179,
       208034.29956726, 139412.59802552, 142693.05405365, 438609.97959547,
       207519.04409889, 113752.98480648, 135839.84697356, 132654.79527399,
       138526.29048876, 129760.27085996, 248744.84587959, 160834.03169376,
       200150.3307942 , 200820.57263921, 119438.47718549, 110537.45038183,
       299248.6480769 , 293688.60902285, 201195.6802444 , 156157.04874775,
       177336.6858616 , 124335.60942132, 172147.73879779, 137930.74762891,
       138949.33632438, 122088.13830159, 142276.87034746, 116766.97113289,
       147250.56433472, 129062.65846659,  84592.13150179, 138282.93453943,
       171163.82270902, 192403.80065236, 143074.04132616, 231154.14604223,
       247191.52310547, 164436.87598152, 304373.55919368, 164741.71939024,
        98435.42027   , 230280.55626613, 135130.21742971, 188772.82643333,
       216052.21309267, 184478.89235053, 117281.06954661, 120990.50119284,
       147472.20529456, 1

## How about a Random Forest

In [40]:
from sklearn.ensemble import RandomForestRegressor

In [41]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
rfr.score(X_test, y_test)

0.8801421787470414

### Let's do some parameter tuning

In [42]:
params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [12, 14, 16],
    'min_samples_leaf': [4, 5],
    'min_samples_split': [4, 5],
    'max_leaf_nodes': [500, 600, 700],
    'max_samples': [None],
    
}
grid = GridSearchCV(rfr, params, refit=True, verbose=3, n_jobs=-1)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


0.8796918816481072

In [43]:
grid.best_params_

{'max_depth': 12,
 'max_leaf_nodes': 500,
 'max_samples': None,
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 100}

In [44]:
rfrPred = inv_boxcox(pd.Series(rfr.predict(X_test)), y_trainLambda)
rfrPred

0      166495.420194
1      173168.552345
2      251504.006118
3      142223.139272
4      199076.538107
           ...      
428    111400.293032
429    198748.663978
430    159724.937318
431    125735.846250
432     96414.917704
Length: 433, dtype: float64

## XG Boost?

In [45]:
import xgboost as xgb

In [46]:
logitXB = xgb.XGBRegressor()
logitXB.fit(X_train, y_train)
logitXB.score(X_test, y_test)

0.8829571285016263

In [47]:
from sklearn.metrics import mean_absolute_error

In [48]:
mean_train = np.mean(y_train)
mean_train[0]
mean_test = np.mean(y_test)

In [49]:
baseline_predictions = np.ones(y_test.shape) * mean_train[0]
baseline_predictions

array([[8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.86915444],
       [8.869

In [50]:
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print(f'Baseline MAE is {round(mae_baseline,2)}')

Baseline MAE is 0.16


In [51]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [52]:
params['eval_metric'] = "mae"

In [53]:
num_boost_round = 999

In [54]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [55]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[11:17:32] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:5.85206
[1]	Test-mae:4.09553
[2]	Test-mae:2.86725
[3]	Test-mae:2.00971
[4]	Test-mae:1.40812
[5]	Test-mae:0.98539
[6]	Test-mae:0.69086
[7]	Test-mae:0.48468
[8]	Test-mae:0.34235
[9]	Test-mae:0.24173
[10]	Test-mae:0.17292
[11]	Test-mae:0.12680
[12]	Test-mae:0.09687
[13]	Test-mae:0.07920
[14]	Test-mae:0.06984
[15]	Test-mae:0.06393
[16]	Test-mae:0.06052
[17]	Test-mae:0.05829
[18]	Test-mae:0.05697
[19]	Test-mae:0.05658
[20]	Test-mae:0.05594
[21]	Test-mae:0.05510
[22]	Test-mae:0.05456
[23]	Test-mae:0.05471
[24]	Test-mae:0.05447
[25]	Test-mae:0.05390
[26]	Test-mae:0.05373
[27]	Test-mae:0.05376
[28]	Test-mae:0.05356
[29]	Test-mae:0.05344
[30]	Test-mae:0.05350
[31]	Test-mae:0.05342
[32]	Test-mae:0.05332
[33]	Test-mae:0.05333
[34]	Test-mae:0.05351
[35]	Test-mae:0.05332
[36]	Test-mae:0.05307
[37]	Test-mae:0.05304
[38]	Test-mae:0.052

In [57]:
print(f'Best MAE is {round(model.best_score, 2)} with {round(model.best_iteration+1, 2)} rounds')

Best MAE is 0.05 with 86 rounds


In [58]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

[11:20:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:20:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:20:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:20:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:20:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,5.861513,0.004560,5.861474,0.028056
1,4.105235,0.003193,4.105196,0.029424
2,2.875188,0.002235,2.875149,0.030381
3,2.014749,0.001578,2.013750,0.028135
4,1.411831,0.001092,1.411659,0.024481
...,...,...,...,...
74,0.005463,0.000342,0.057107,0.003149
75,0.005356,0.000333,0.057089,0.003149
76,0.005217,0.000319,0.057092,0.003207
77,0.005111,0.000341,0.057087,0.003209


In [59]:
cv_results['test-mae-mean'].min()

0.0570704

In [60]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(8,13)
    for min_child_weight in range(5,10)
]

In [61]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=8, min_child_weight=5
[11:21:11] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:11] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:11] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:11] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:11] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 0.0559832 for 42 rounds
CV with max_depth=8, min_child_weight=6
[11:21:14] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now de

	MAE 0.0570634 for 24 rounds
CV with max_depth=10, min_child_weight=5
[11:21:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 0.057193400000000005 for 33 rounds
CV with max_depth=10, min_child_weight=6
[11:21:37] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\r

	MAE 0.056600399999999995 for 23 rounds
CV with max_depth=12, min_child_weight=5
[11:21:59] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:59] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:59] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:59] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:59] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 0.056847400000000006 for 22 rounds
CV with max_depth=12, min_child_weight=6
[11:22:01] WARNING: c:\ci\xgboost-split_1638290375667\work\src\

In [62]:
params['max_depth'] = 8
params['min_child_weight'] = 7

In [63]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [64]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[11:22:43] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:22:43] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:22:43] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:22:43] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:22:43] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 0.055550600000000006 for 42 rounds
CV with subsample=1.0, colsample=0.9
[11:22:46] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is n

	MAE 0.05738620000000001 for 26 rounds
CV with subsample=0.8, colsample=0.8
[11:23:04] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:04] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:04] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:04] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:04] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 0.0566296 for 33 rounds
CV with subsample=0.8, colsample=0.7
[11:23:06] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression

In [65]:
params['subsample'] = 1
params['colsample_bytree'] = 1

In [66]:
%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time cv_results = xgb.cv(params, dtrain,num_boost_round=num_boost_round,seed=42,nfold=5, metrics=['mae'], early_stopping_rounds=10)    
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

Wall time: 0 ns
CV with eta=0.3
[11:23:31] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:31] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:31] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:31] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:31] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
Wall time: 2.57 s
	MAE 0.055550600000000006 for 42 rounds

CV with eta=0.2
[11:23:33] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now depr

In [67]:
params['eta'] = .05

In [68]:
num_boost_round

999

In [69]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[11:25:29] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:7.94228
[1]	Test-mae:7.54511
[2]	Test-mae:7.16779
[3]	Test-mae:6.80931
[4]	Test-mae:6.46874
[5]	Test-mae:6.14518
[6]	Test-mae:5.83779
[7]	Test-mae:5.54575
[8]	Test-mae:5.26829
[9]	Test-mae:5.00470
[10]	Test-mae:4.75427
[11]	Test-mae:4.51635
[12]	Test-mae:4.29031
[13]	Test-mae:4.07588
[14]	Test-mae:3.87272
[15]	Test-mae:3.67930
[16]	Test-mae:3.49587
[17]	Test-mae:3.32165
[18]	Test-mae:3.15567
[19]	Test-mae:2.99843
[20]	Test-mae:2.84880
[21]	Test-mae:2.70671
[22]	Test-mae:2.57153
[23]	Test-mae:2.44312
[24]	Test-mae:2.32104
[25]	Test-mae:2.20521
[26]	Test-mae:2.09509
[27]	Test-mae:1.99055
[28]	Test-mae:1.89101
[29]	Test-mae:1.79655
[30]	Test-mae:1.70690
[31]	Test-mae:1.62181
[32]	Test-mae:1.54104
[33]	Test-mae:1.46424
[34]	Test-mae:1.39135
[35]	Test-mae:1.32177
[36]	Test-mae:1.25589
[37]	Test-mae:1.19315
[38]	Test-mae:1.133

In [70]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[11:25:43] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:7.94228
[1]	Test-mae:7.54511
[2]	Test-mae:7.16779
[3]	Test-mae:6.80931
[4]	Test-mae:6.46874
[5]	Test-mae:6.14518
[6]	Test-mae:5.83779
[7]	Test-mae:5.54575
[8]	Test-mae:5.26829
[9]	Test-mae:5.00470
[10]	Test-mae:4.75427
[11]	Test-mae:4.51635
[12]	Test-mae:4.29031
[13]	Test-mae:4.07588
[14]	Test-mae:3.87272
[15]	Test-mae:3.67930
[16]	Test-mae:3.49587
[17]	Test-mae:3.32165
[18]	Test-mae:3.15567
[19]	Test-mae:2.99843
[20]	Test-mae:2.84880
[21]	Test-mae:2.70671
[22]	Test-mae:2.57153
[23]	Test-mae:2.44312
[24]	Test-mae:2.32104
[25]	Test-mae:2.20521
[26]	Test-mae:2.09509
[27]	Test-mae:1.99055
[28]	Test-mae:1.89101
[29]	Test-mae:1.79655
[30]	Test-mae:1.70690
[31]	Test-mae:1.62181
[32]	Test-mae:1.54104
[33]	Test-mae:1.46424
[34]	Test-mae:1.39135
[35]	Test-mae:1.32177
[36]	Test-mae:1.25589
[37]	Test-mae:1.19315
[38]	Test-mae:1.133

In [71]:
mean_absolute_error(best_model.predict(dtest), y_test)

0.048077063701883326

In [72]:
params

{'max_depth': 8,
 'min_child_weight': 7,
 'eta': 0.05,
 'subsample': 1,
 'colsample_bytree': 1,
 'objective': 'reg:linear',
 'eval_metric': 'mae'}

In [73]:
best_model.save_model("my_model.model")

In [74]:
loaded_model = xgb.Booster()
loaded_model.load_model("my_model.model")
loaded_model.predict(dtest)

[11:26:07] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


array([8.853082 , 8.9144945, 9.105281 , 8.821158 , 8.852187 , 8.761427 ,
       8.758885 , 9.398232 , 8.950732 , 8.698443 , 8.7826395, 8.7506695,
       8.91215  , 8.7276745, 9.146851 , 8.738885 , 9.176173 , 8.92558  ,
       8.647757 , 8.597634 , 9.215286 , 9.105963 , 9.083826 , 8.715002 ,
       8.930911 , 8.867996 , 8.96843  , 8.779917 , 8.893547 , 8.694055 ,
       8.911138 , 8.660637 , 8.810259 , 8.78809  , 8.518305 , 8.651907 ,
       8.854659 , 8.952916 , 8.834629 , 9.049654 , 9.1122875, 8.942877 ,
       9.220032 , 8.894286 , 8.5346775, 9.04816  , 8.668804 , 8.997986 ,
       8.810518 , 8.858234 , 8.707584 , 8.820728 , 8.922127 , 8.686567 ,
       9.0721655, 8.713883 , 8.718867 , 9.053779 , 8.755601 , 8.366441 ,
       8.519272 , 8.59969  , 8.896974 , 8.8974905, 8.966882 , 8.665894 ,
       9.080928 , 8.909896 , 9.275048 , 8.959674 , 9.357043 , 9.003739 ,
       8.916028 , 9.004841 , 9.2053385, 8.929986 , 8.975197 , 8.5107155,
       8.681485 , 8.699932 , 8.64306  , 8.899713 , 

In [75]:
RSquared = 1 - (mean_absolute_error(best_model.predict(dtest), y_test)/mean_test[0])

In [76]:
RSquared

0.9945738889789107

# Let's Put all our Models together for some Ensembling 

In [77]:
FullPredicitions = pd.concat([inv_boxcox(pd.Series(loaded_model.predict(dtest)), y_trainLambda), inv_boxcox(y_test.SalePrice, y_trainLambda)], axis = 1)
FullPredicitions.columns = ['XGBoost', 'Actual']

In [78]:
FullPredictions = pd.concat([pd.Series(LassoPred), FullPredicitions], axis = 1)
FullPredictions.columns = ['Lasso','XGBoost', 'Actual']
FullPredictions

,Lasso,XGBoost,Actual
0,150648.669690,162087.156250,173000.0
1,215524.052500,182213.421875,135000.0
2,236758.911064,263364.375000,267000.0
3,149574.239902,152563.515625,161000.0
4,208034.299567,161811.921875,136000.0
...,...,...,...
428,124465.370659,114405.867188,135750.0
429,187756.483967,187597.906250,192500.0
430,156978.257184,154605.453125,146500.0
431,137367.491672,128744.328125,118500.0


In [79]:
FullPredictions = pd.concat([PredGrid.Predicted, FullPredictions], axis = 1)
FullPredictions.columns = ['ElasticNet','Lasso','XGBoost', 'Actual']
FullPredictions

,ElasticNet,Lasso,XGBoost,Actual
0,175678.225969,150648.669690,162087.156250,173000.0
1,177164.158485,215524.052500,182213.421875,135000.0
2,253768.675265,236758.911064,263364.375000,267000.0
3,146824.620062,149574.239902,152563.515625,161000.0
4,151406.096205,208034.299567,161811.921875,136000.0
...,...,...,...,...
428,115837.427273,124465.370659,114405.867188,135750.0
429,194479.420415,187756.483967,187597.906250,192500.0
430,159433.946958,156978.257184,154605.453125,146500.0
431,127467.008067,137367.491672,128744.328125,118500.0


In [80]:
FullPredictions = pd.concat([rfrPred, FullPredictions], axis = 1)
FullPredictions.columns = ['RandomForest','ElasticNet','Lasso','XGBoost', 'Actual']
FullPredictions

,RandomForest,ElasticNet,Lasso,XGBoost,Actual
0,166495.420194,175678.225969,150648.669690,162087.156250,173000.0
1,173168.552345,177164.158485,215524.052500,182213.421875,135000.0
2,251504.006118,253768.675265,236758.911064,263364.375000,267000.0
3,142223.139272,146824.620062,149574.239902,152563.515625,161000.0
4,199076.538107,151406.096205,208034.299567,161811.921875,136000.0
...,...,...,...,...,...
428,111400.293032,115837.427273,124465.370659,114405.867188,135750.0
429,198748.663978,194479.420415,187756.483967,187597.906250,192500.0
430,159724.937318,159433.946958,156978.257184,154605.453125,146500.0
431,125735.846250,127467.008067,137367.491672,128744.328125,118500.0


In [81]:
meltTemp = pd.melt(FullPredictions, id_vars='Actual')
meltTemp
#fig = px.scatter(meltTemp.loc[meltTemp['variable'].isin(['RandomForest', 'Lasso', 'ElasticNet','XGBoost'])], x = 'Actual', y = 'value', color = 'variable')
#fig


,Actual,variable,value
0,173000.0,RandomForest,166495.420194
1,135000.0,RandomForest,173168.552345
2,267000.0,RandomForest,251504.006118
3,161000.0,RandomForest,142223.139272
4,136000.0,RandomForest,199076.538107
...,...,...,...
1727,135750.0,XGBoost,114405.867188
1728,192500.0,XGBoost,187597.906250
1729,146500.0,XGBoost,154605.453125
1730,118500.0,XGBoost,128744.328125


In [82]:
FullPredictions['RandomForest_Variance'] = np.sqrt((FullPredictions['RandomForest'] - FullPredictions['Actual'])**2)
FullPredictions['ElasticNet_Variance'] = np.sqrt((FullPredictions['ElasticNet'] - FullPredictions['Actual'])**2)
FullPredictions['Lasso_Variance'] = np.sqrt((FullPredictions['Lasso'] - FullPredictions['Actual'])**2)
FullPredictions['XGBoost_Variance'] = np.sqrt((FullPredictions['XGBoost'] - FullPredictions['Actual'])**2)
FullPredictions

,RandomForest,ElasticNet,Lasso,XGBoost,Actual,RandomForest_Variance,ElasticNet_Variance,Lasso_Variance,XGBoost_Variance
0,166495.420194,175678.225969,150648.669690,162087.156250,173000.0,6504.579806,2678.225969,22351.330310,10912.843750
1,173168.552345,177164.158485,215524.052500,182213.421875,135000.0,38168.552345,42164.158485,80524.052500,47213.421875
2,251504.006118,253768.675265,236758.911064,263364.375000,267000.0,15495.993882,13231.324735,30241.088936,3635.625000
3,142223.139272,146824.620062,149574.239902,152563.515625,161000.0,18776.860728,14175.379938,11425.760098,8436.484375
4,199076.538107,151406.096205,208034.299567,161811.921875,136000.0,63076.538107,15406.096205,72034.299567,25811.921875
...,...,...,...,...,...,...,...,...,...
428,111400.293032,115837.427273,124465.370659,114405.867188,135750.0,24349.706968,19912.572727,11284.629341,21344.132813
429,198748.663978,194479.420415,187756.483967,187597.906250,192500.0,6248.663978,1979.420415,4743.516033,4902.093750
430,159724.937318,159433.946958,156978.257184,154605.453125,146500.0,13224.937318,12933.946958,10478.257184,8105.453125
431,125735.846250,127467.008067,137367.491672,128744.328125,118500.0,7235.846250,8967.008067,18867.491672,10244.328125


In [83]:
ScoreDF = pd.DataFrame({
    'RandomForestMAE': [FullPredictions.RandomForest_Variance.sum()/(len(FullPredictions.index))],
    'ElasticNetMAE': [FullPredictions.ElasticNet_Variance.sum()/(len(FullPredictions.index))],
    'LassoMAE': [FullPredictions.Lasso_Variance.sum()/(len(FullPredictions.index))],
    'XGBoostMAE': [FullPredictions.XGBoost_Variance.sum()/(len(FullPredictions.index))]
})
ScoreDF

    

,RandomForestMAE,ElasticNetMAE,LassoMAE,XGBoostMAE
0,17568.248776,16718.181917,29174.547791,16038.985611


In [120]:
ScoreDF['RandomForestRSquared'] = 1-(ScoreDF.RandomForestMAE/inv_boxcox(mean_test[0], y_trainLambda))
ScoreDF['ElasticNetRSquared'] = 1-(ScoreDF.ElasticNetMAE/inv_boxcox(mean_test[0], y_trainLambda))
ScoreDF['LassoRSquared'] = 1-(ScoreDF.LassoMAE/inv_boxcox(mean_test[0], y_trainLambda))
ScoreDF['XGBoostRSquared'] = 1-(ScoreDF.XGBoostMAE/inv_boxcox(mean_test[0], y_trainLambda))



In [121]:
ScoreDF

,RandomForestMAE,ElasticNetMAE,LassoMAE,XGBoostMAE,RandomForestRSquared,ElasticNetRSquared,LassoRSquared,XGBoostRSquared
0,17568.248776,16718.181917,29174.547791,16038.985611,0.893093,0.898266,0.822465,0.902399


In [86]:
EnsembleDF = FullPredictions.loc[:,['RandomForest', 'Lasso', 'XGBoost', 'ElasticNet', 'Actual']]

loopDF = FullPredictions.loc[:,['Actual']]
minMAE = float('Inf')
pLst = []
for i in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
    for j in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
        for k in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
            for m in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
                if (i + j + k + m) == 1:
                    loopDF['RandomForest'] = FullPredictions.RandomForest * i
                    loopDF['Lasso'] = FullPredictions.Lasso * j
                    loopDF['XGBoost'] = FullPredictions.XGBoost * k
                    loopDF['ElasticNet'] = FullPredictions.ElasticNet * m
                    loopDF['Prediction'] = loopDF['ElasticNet'] + loopDF['XGBoost'] + loopDF['Lasso'] + loopDF['RandomForest']
                    loopDF['Difference'] = loopDF['Prediction'] - loopDF['Actual']
                    loopDF['Diffsq'] = np.sqrt(loopDF['Difference']**2)
                    mae = loopDF.Diffsq.sum()/len(loopDF.index)
                    if mae < minMAE:
                        minMAE = mae
                        pLst = [i,j,k,m]
                        print(mae)


                        
        
    
    

17568.248775582786
17221.47758968276
17166.134999169382
16921.172522503548
16842.59301262325
16821.728122558077
16664.333214842907
16541.628755737518
16462.07430060207
16322.243334579167
16300.552601691148
16281.442310320135
16187.592918776409
16124.426494433017
16092.965530292404
16077.943281489943
16066.197993548205
16024.120921587395
15953.003883611444
15912.01214185293
15905.474492052723
15880.690668652414
15812.216541677251
15758.072934987704
15734.558520205044
15719.613634642006
15655.981721738304
15624.16376771257
15598.999739767021
15570.78484993038
15556.465777315681


In [87]:
pLst

[0, 0, 0.6, 0.4]

In [88]:
minMAE

15556.465777315681

# Ensembling Complete:
### RandomForest = 0.0
### Lasso = 0.0
### XGBoost = 0.6
### ElasticNet = 0.4

RandomState 0 gives MAE of 15,147m

RandomState 1 gives MAE of 15,556

# Let's Predict

In [89]:
# Reorder Columns in finalDF to match X_train
finalDF = finalDF[X_train.columns]

In [90]:
# Lasso Predicitons
finLasPred = logit.predict(finalDF)

In [91]:
# RandomForest Predictions
finRFRPred = grid.predict(finalDF)

In [92]:
# Create labels DF for XGBoost
df_y = pd.DataFrame(np.zeros((1448,1)))
df_y.columns = ['SalePrice']
df_y

,SalePrice
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1443,0.0
1444,0.0
1445,0.0
1446,0.0


In [93]:
# XGBoost Prediction
finXGBPred = loaded_model.predict(xgb.DMatrix(finalDF, label=df_y))

In [104]:
# ElasticNet Prediction
finENPred = gridEn.predict(finalDF)

## Bring all Predictions together and analyize

In [105]:
FinalPred = pd.concat([inv_boxcox(pd.Series(finLasPred), y_trainLambda), inv_boxcox(pd.Series(finRFRPred), y_trainLambda), inv_boxcox(pd.Series(finXGBPred), y_trainLambda), inv_boxcox(pd.Series(finENPred), y_trainLambda)], axis = 1)
FinalPred.columns = ['Lasso_Predictions', 'RandomForest_Predictions', 'XGBoost_Predictions', 'ElasticNet_Predictions']
FinalPred

,Lasso_Predictions,RandomForest_Predictions,XGBoost_Predictions,ElasticNet_Predictions
0,126312.344812,123275.538161,120938.148438,113565.958246
1,183383.500087,157602.698804,157019.171875,166453.155447
2,167458.860249,192780.266691,188672.703125,192596.885542
3,171685.227229,195920.890423,197923.234375,203945.179961
4,166202.887286,192509.332886,191101.203125,194898.948573
...,...,...,...,...
1443,112743.898034,91300.419168,93875.875000,80338.466318
1444,113353.186576,95383.230172,87712.515625,82201.827036
1445,170028.921529,170536.092576,166348.343750,172319.508832
1446,126764.445325,110262.059659,121637.914062,116097.004991


## Let's Visualize the Deltas of all our predictions against eachother

In [108]:
AA = FinalPred
AA['Delta:Lasso-RanForest'] = AA.RandomForest_Predictions - AA.Lasso_Predictions
AA['Delta:Lasso-XGBoost'] = AA.XGBoost_Predictions - AA.Lasso_Predictions
AA['Delta:Lasso-ElasticNet'] = AA.ElasticNet_Predictions - AA.Lasso_Predictions
AA['Delta:XGBoost-RanForest'] = AA.RandomForest_Predictions - AA.XGBoost_Predictions
AA['Delta:XGBoost-ElasticNet'] = AA.ElasticNet_Predictions - AA.XGBoost_Predictions
AA['Delta:RanForest-ElasticNet'] = AA.ElasticNet_Predictions - AA.RandomForest_Predictions

AA.drop(['Lasso_Predictions', 'RandomForest_Predictions', 'XGBoost_Predictions', 'ElasticNet_Predictions'], axis =1, inplace = True)
AAmelt = pd.melt(AA)
AAmelt



,variable,value
0,Delta:Lasso-RanForest,-3036.806651
1,Delta:Lasso-RanForest,-25780.801282
2,Delta:Lasso-RanForest,25321.406443
3,Delta:Lasso-RanForest,24235.663194
4,Delta:Lasso-RanForest,26306.445600
...,...,...
8683,Delta:RanForest-ElasticNet,-10961.952850
8684,Delta:RanForest-ElasticNet,-13181.403136
8685,Delta:RanForest-ElasticNet,1783.416256
8686,Delta:RanForest-ElasticNet,5834.945332


In [122]:
fig = px.box(AAmelt, y = 'value', x = 'variable', color_discrete_sequence = ['#454b59'], title = 'Difference Between 3 Predictive Models')
fig.update_layout(
    xaxis_title = 'Model',
    yaxis_title = 'Delta Sale Price ($)',
    title_x=0.5,
    plot_bgcolor = '#d0c5b5',
    title_font = dict(size = 25),
    yaxis = dict(
        tickfont = dict(size=14),
        titlefont = dict(size = 25),
        showline = True,
        linecolor = 'black',
        range = [-150000,150000]
    ),
    xaxis = dict(
        tickfont = dict(size=16),
        titlefont = dict(size =25),

        linecolor = 'black'        
    )    
)
for t in fig.data:
    t.marker.line.width = 1
    t.marker.line.color = "black"
fig

## Random Forest and XGBoost agree with each other the most
**Will use 60% XGBoost and 40% RandomForest**

In [111]:
FinalPred = pd.concat([inv_boxcox(pd.Series(finLasPred), y_trainLambda), inv_boxcox(pd.Series(finRFRPred), y_trainLambda), inv_boxcox(pd.Series(finXGBPred), y_trainLambda), inv_boxcox(pd.Series(finENPred), y_trainLambda)], axis = 1)
FinalPred.columns = ['Lasso_Predictions', 'RandomForest_Predictions', 'XGBoost_Predictions', 'ElasticNet_Predictions']
FinalPredictions = (0.6 * FinalPred['XGBoost_Predictions']) + (.4* FinalPred['ElasticNet_Predictions'])

## Let's look at the distribution between my predicitions and the given dataset

In [114]:
BB = pd.DataFrame(FinalPredictions)
BB['TrainSalePrice'] = inv_boxcox(y_train['SalePrice'], y_trainLambda)
BB.columns = ['Predicted Sale Price', 'Disclosed Sale Price']
BBmelt = pd.melt(BB)
BBmelt

,variable,value
0,Predicted Sale Price,117989.273924
1,Predicted Sale Price,160792.769991
2,Predicted Sale Price,190242.379217
3,Predicted Sale Price,200332.017297
4,Predicted Sale Price,192620.305992
...,...,...
2891,Disclosed Sale Price,NaN
2892,Disclosed Sale Price,NaN
2893,Disclosed Sale Price,NaN
2894,Disclosed Sale Price,NaN


In [115]:
fig = px.box(BBmelt, y = 'value', x = 'variable', color_discrete_sequence = ['#454b59'], title = 'Distribution of Predicted and Disclosed Sale Prices')
fig.update_layout(
    xaxis_title = 'Type',
    yaxis_title = 'Sale Price ($)',
    title_x=0.5,
    plot_bgcolor = '#d0c5b5',
    title_font = dict(size = 25),
    yaxis = dict(
        tickfont = dict(size=14),
        titlefont = dict(size = 25),
        showline = True,
        linecolor = 'black'
       
    ),
    xaxis = dict(
        tickfont = dict(size=16),
        titlefont = dict(size =25),

        linecolor = 'black'        
    )    
)
for t in fig.data:
    t.marker.line.width = 1
    t.marker.line.color = "black"
fig

**distributions look pretty similar and are centered pretty close to each other**